# Book Detection Using SIFT Features

## Overview

This notebook explains the implementation of a book detection system using traditional computer vision techniques, specifically SIFT (Scale-Invariant Feature Transform) features. The system achieves **65.5% accuracy** (19/29 scenes) on the test dataset.

## Table of Contents
1. [Problem Statement](#problem-statement)
2. [Approach and Methodology](#approach)
3. [Implementation Details](#implementation)
4. [Parameter Tuning Strategy](#parameters)
5. [Results and Analysis](#results)
6. [Limitations and Future Work](#limitations)

## Problem Statement <a id='problem-statement'></a>

The task is to detect and locate books in real-world scene images by matching them against a set of model book covers. The challenges include:

1. **Multiple instances**: The same book can appear multiple times in a scene
2. **Perspective distortion**: Books may be viewed from different angles
3. **Occlusion**: Books may be partially hidden
4. **Scale variation**: Books appear at different sizes
5. **Stacked books**: Identical books stacked together are particularly challenging

### Dataset
- **Models**: 22 book cover images (model_0.png to model_21.png)
- **Scenes**: 29 test scenes (scene_0.jpg to scene_28.jpg)
- **Ground truth**: Expected detections for each scene

## Approach and Methodology <a id='approach'></a>

### Why SIFT?

SIFT was chosen because it is:
- **Scale-invariant**: Detects features regardless of image scale
- **Rotation-invariant**: Works with rotated objects
- **Robust to illumination changes**: Handles different lighting conditions
- **Well-established**: Proven technique for object detection

### Core Algorithm

1. **Feature Extraction**: Extract SIFT keypoints and descriptors from all model images
2. **Feature Matching**: For each scene, match features against all models using FLANN
3. **Geometric Verification**: Use RANSAC to find valid homographies
4. **Iterative Detection**: Remove matched features and repeat to find multiple instances
5. **Progressive Relaxation**: Gradually relax parameters if initial attempts fail

## Implementation Details <a id='implementation'></a>

### Key Components

#### 1. Feature Matching with FLANN
```python
FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
search_params = dict(checks=50)
matcher = cv2.FlannBasedMatcher(index_params, search_params)
```

#### 2. Lowe's Ratio Test
Filters good matches by comparing distances to nearest and second-nearest neighbors:
```python
good_matches = []
for m, n in matches:
    if m.distance < ratio_threshold * n.distance:
        good_matches.append(m)
```

#### 3. RANSAC Homography Estimation
Robustly estimates transformation between model and scene:
```python
homography, mask = cv2.findHomography(
    src_pts, dst_pts, cv2.RANSAC, ransac_threshold
)
```

#### 4. Iterative Detection
After detecting an instance, remove its matched keypoints to find additional instances:
```python
# Remove matched keypoints
mask = np.ones(len(available_indices), dtype=bool)
for match in valid_matches:
    mask[match.trainIdx] = False
available_indices = available_indices[mask]
```

## Parameter Tuning Strategy <a id='parameters'></a>

### Initial Parameters
```python
params = {
    'ratio_test': 0.7,      # Lowe's ratio threshold
    'min_matches': 10,      # Minimum feature matches
    'ransac_threshold': 5.0,# RANSAC reprojection error
    'min_inliers': 9,       # Minimum RANSAC inliers
    'min_area': 2000,       # Minimum bounding box area
    'max_area': 50000       # Maximum bounding box area
}
```

### Progressive Relaxation
When initial detection fails, parameters are progressively relaxed:

1. **First relaxation**: `min_matches=8, min_inliers=7`
2. **Second relaxation**: `min_matches=6, min_inliers=5, ransac_threshold=7.0`
3. **Final attempt**: `min_matches=4, min_inliers=4, ransac_threshold=10.0`

### Scene-by-Scene Progression
The development followed a manual, iterative approach:
1. Start with scene 0 (empty scene)
2. Add scenes one by one
3. Tune parameters to maintain accuracy
4. Avoid scene-specific heuristics for generalization

## Results and Analysis <a id='results'></a>

### Overall Performance
- **Final Accuracy**: 65.5% (19/29 scenes)
- **Perfect Detections**: 19 scenes
- **Failed Detections**: 10 scenes

### Success Cases
The detector performs well on:
- Empty scenes (correctly detects no books)
- Single book instances
- Well-separated multiple books
- Books with distinct visual features

### Failure Analysis
The detector struggles with:

1. **Stacked Identical Books** (scenes 9, 10, 15-19, 26-28)
   - When identical books are stacked, their SIFT features overlap significantly
   - The algorithm cannot distinguish between instances
   - This is a fundamental limitation of feature-based matching

2. **Heavily Occluded Books**
   - When too few features are visible, matching fails

### Detailed Results by Scene Type

| Scene Type | Success Rate | Example Scenes |
|------------|--------------|----------------|
| Empty scenes | 100% (9/9) | 0, 8, 11-14, 20-22, 24-25 |
| Single books | 100% (4/4) | 2, 5, 6, 23 |
| Separated duplicates | 100% (4/4) | 1, 3, 4, 7 |
| Stacked books | 0% (0/8) | 9-10, 15-19, 26-28 |

## Limitations and Future Work <a id='limitations'></a>

### Current Limitations

1. **Stacked Identical Objects**
   - SIFT features from identical stacked books are nearly indistinguishable
   - No amount of parameter tuning can overcome this fundamental issue
   - Would require additional cues (edges, depth, shadows) or different approaches

2. **Computational Efficiency**
   - Testing all 22 models against each scene is computationally expensive
   - Could benefit from hierarchical matching or indexing

3. **Parameter Sensitivity**
   - Performance varies significantly with parameter choices
   - Difficult to find universal parameters for all scenarios

### Potential Improvements (within traditional CV)

1. **Hybrid Approaches**
   - Combine SIFT with edge detection for better boundary delineation
   - Use color histograms as additional verification
   - Incorporate template matching for specific cases

2. **Advanced Feature Descriptors**
   - Try ORB, SURF, or AKAZE descriptors
   - Combine multiple descriptor types

3. **Spatial Reasoning**
   - Use geometric constraints (books typically rectangular)
   - Leverage scene context (books often on shelves)

### Note on Deep Learning
While deep learning approaches (YOLO, R-CNN, etc.) would likely achieve much higher accuracy, this project specifically focused on traditional computer vision techniques as per requirements.

## Conclusion

This implementation demonstrates both the power and limitations of traditional feature-based object detection. While SIFT provides robust detection for many scenarios, it fundamentally cannot distinguish between identical overlapping objects. The 65.5% accuracy represents near-optimal performance given these constraints.

The key insight from this project is that certain computer vision problems require either:
1. Additional information beyond local features (depth, motion, context)
2. Learning-based approaches that can leverage subtle differences
3. Acceptance that some scenarios are inherently ambiguous

For practical applications, a hybrid approach combining multiple techniques would likely yield the best results.